In [7]:
import numpy as np
from keras import models
from keras import layers
import pandas as pd
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error

# Set random seed
np.random.seed(0)
number_of_features = 7

In [16]:
#china dataset that doesn't work
df = pd.read_csv("preproc.csv") 
train, test = train_test_split(df, test_size=0.2)
train = train.drop(['Date','1d'], axis = 1)
test = test.drop(['Date','1d'], axis = 1)

train_y = train[train.columns[1]].values
train_x = train.drop(['PM2.5'], axis = 1)

test_y = test[test.columns[1]].values
test_x = test.drop(['PM2.5'], axis = 1).values

normalizer = MinMaxScaler(feature_range = (0, 1))
temp = normalizer.fit_transform(train_x)
temp1 = normalizer.fit_transform(test_x)

#train_y = train_y.reshape(-1, 1)
#train_y = normalizer.fit_transform(train_y)
#test_y = test_y.reshape(-1, 1)
#test_y = normalizer.fit_transform(test_y)

train_x.loc[:,:] = temp
test_x.loc[:,:] = train_x.loc[:,:] = temp


,DE1P,PRES,RAIN,TEMP,1SPM,index,station
4663,0.875221,0.358935,0.000000,0.807271,0.188256,0.280587,0.090909
7430,0.693158,0.134093,0.000000,0.718365,0.207830,0.441616,0.454545
12207,0.762279,0.523489,0.000000,0.652871,0.146297,0.719579,0.727273
14026,0.835960,0.376179,0.011742,0.724245,0.158594,0.824368,0.818182
4898,0.726769,0.360926,0.000000,0.761631,0.362694,0.294847,0.090909
...,...,...,...,...,...,...,...
6495,0.811312,0.283043,0.008102,0.654485,0.172712,0.386798,0.000000
1275,0.800201,0.377237,0.000000,0.784491,0.242372,0.073239,0.636364
1936,0.392597,0.667479,0.000000,0.386468,0.154289,0.118873,0.545455
13452,0.434904,0.471815,0.000000,0.310454,0.189983,0.791397,0.818182


In [9]:
def create_network(neurons = 1, lr = 0.0001):
    
    # Start neural network
    network = models.Sequential()

    # Add first hidden layer with a ReLU activation function
    network.add(layers.Dense(neurons, activation='relu', input_shape=(number_of_features,)))

    # Add second hidden layer with a ReLU activation function
    network.add(layers.Dense(neurons, activation='relu'))

    # Add final layer with a linear activation function
    network.add(layers.Dense(units=1), activation='linear')

    # Compile neural network
    network.compile(loss='mean_squared_error', optimizer='adam')
    
    # Return compiled network
    return network

In [10]:
neural_network = KerasRegressor(build_fn=create_network, verbose=0)

In [11]:
lr = [0.0001, 0.001, 0.01, 0.1, 0.3, 1, 0.2]
neurons = [1, 5, 10, 15, 20, 25, 30,50,100,200]

hyperparameters = dict(lr=lr, neurons = neurons)


In [12]:
# Create grid search
grid = GridSearchCV(estimator=neural_network, cv=3, param_grid=hyperparameters)

# Fit grid search
grid_result = grid.fit(train_x, train_y)

KeyboardInterrupt: 

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))